In [ ]:
'''Objective: Find a specific board's boot success over time
              Also track boot time over time'''
import pandas as pd
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
"""Get boot reports"""
import requests
from urlparse import urljoin
import kernelci_api_key

BACKEND_URL = "http://api.kernelci.org"
JOB = 'mainline'
BOARD = 'am335x-boneblack'
DEFCONFIG = 'multi_v7_defconfig'
DATE_RANGE = 200
STORAGE_SERVER = 'http://storage.kernelci.org'

def invoke():
    headers = { "Authorization": kernelci_api_key.getkernelcikey()}
    params = {
        "job": JOB,
        "board": BOARD,
        "defconfig":DEFCONFIG,
#        "date_range": DATE_RANGE
    }
    url = urljoin(BACKEND_URL, "/boot")
    response = requests.get(url, params=params, headers=headers)
    return response.content

In [ ]:
content = invoke()
content

In [ ]:
contentjs = json.loads(content)
df = json_normalize(contentjs['result'])
df.head()

In [ ]:
df.columns

In [ ]:
df2 = df[[u'_id.$oid',u'boot_result_description',u'created_on.$date',u'defconfig',u'git_describe',u'lab_name',u'status',u'time.$date',u'warnings']]
df2.head()

In [ ]:
df2.columns

In [ ]:
df2.columns = [u'boot_id',u'boot_result_description', u'created_on', u'defconfig', u'git_describe', u'lab_name', u'status',u'boot_time',u'warnings']
df2.head()

In [ ]:
df2['created_on'] = pd.to_datetime(df2['created_on'],unit='ms')
df2.head()

In [ ]:
df2.describe(include='all')

In [ ]:
df2.index = df2.created_on
df2.head()

In [ ]:
df2 = df2.sort_index()

In [ ]:
df2.status.value_counts()

In [ ]:
df2 = df2[(df2.status == 'PASS') | (df2.status == 'FAIL')]
df2.status.value_counts()

In [ ]:
df2.head()

In [ ]:
df2['status_fl'] = df2.status.apply(lambda x: 1 if x=='PASS' else 0)
df2.head()

In [ ]:
df2['period'] = df2.created_on.apply(lambda x: pd.Period(x, 'W'))
df2.head()

In [ ]:
df2 = df2.groupby('period').mean()
df2

In [ ]:
df2.status_fl.plot()
plt.ylim(0.4,1)

In [ ]:
df2.boot_time.plot()